## Initializations and Dataset Overview

In [2]:
import sys
from pathlib import Path

# Get the project root (assuming notebook is in a subfolder of the project)
notebook_dir = Path().resolve()
project_root = notebook_dir.parent  # Adjust .parent depth based on your structure

# Add to path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


## Imports

Let's start by getting all our imports. Keep in mind that PyTorch does not automatically detect and train on GPU, you have to tell it to use cuda. In case you want to train on Mac Silicon, replace cuda with mps.

In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam  # Adam Optimizer
from torch.utils.data import Dataset, DataLoader  # Dataset class and DataLoader for creating the objects
from torchinfo import summary  # Visualize the model layers and number of parameters

from sklearn.model_selection import train_test_split  # Split the dataset (train, validation, test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # Regression metrics
import matplotlib.pyplot as plt  # Plotting the training progress at the end
import pandas as pd  # Data reading and preprocessing
import numpy as np  # Mathematical operations
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print(device)

cpu


## EDA & Preprocessing

In this section, you'll perform exploratory data analysis and preprocessing. This includes:
- Loading the Indonesian house prices dataset
- Checking for missing values and handling them
- Analyzing the distribution of the target variable (price_usd)
- Exploring correlations between features
- Handling categorical variables (encoding)
- Detecting and handling outliers
- Feature scaling/normalization

Key things to consider for regression:
- The target variable (price_usd) is continuous, not categorical
- You'll need to encode categorical features like city, district, property_type, etc.
- Consider removing or encoding the listing_date column
- Check the distribution of prices - you might want to use log transformation if it's heavily skewed

In [4]:
# Read dataset
raw_df = pd.read_csv("indonesian_house_prices.csv")
df = raw_df.copy()
print("Dataset overview:")
display(df.head())

# Display dataset information
print(f"\nOriginal data shape: {df.shape}")
print(f"\nColumn names and types:")
display(df.dtypes)
print(f"\nMissing values:")
display(df.isnull().sum())

Dataset overview:


,listing_date,city,district,property_type,price_idr,price_usd,land_size_sqm,building_size_sqm,bedrooms,bathrooms,floors,building_age_years,carports,certificate_type,furnishing,has_swimming_pool,has_garden,has_security,distance_to_city_center_km,nearby_facilities_count
0,2025-03-13,Palembang,East,Apartment,960365122,61170,117,117,2,1,2,13,0,SHM (Freehold),Unfurnished,0,1,1,14.3,8
1,2025-08-07,Medan,Central,House,3685078430,234718,149,114,2,1,1,2,1,SHM (Freehold),Unfurnished,0,0,1,1.8,1
2,2025-09-18,Bogor,Central,Townhouse,1782086831,113509,123,82,3,3,2,10,1,HGB (Building Rights),Semi-Furnished,1,1,1,2.4,5
3,2024-07-09,Batam,North,House,4773459488,304042,507,213,2,1,1,12,1,SHM (Freehold),Semi-Furnished,0,1,1,14.1,1
4,2025-01-02,Bali (Denpasar),West,Townhouse,3407557763,217042,385,246,6,6,1,28,0,HGB (Building Rights),Semi-Furnished,0,0,1,10.2,9



Original data shape: (50000, 20)

Column names and types:


listing_date                   object
city                           object
district                       object
property_type                  object
price_idr                       int64
price_usd                       int64
land_size_sqm                   int64
building_size_sqm               int64
bedrooms                        int64
bathrooms                       int64
floors                          int64
building_age_years              int64
carports                        int64
certificate_type               object
furnishing                     object
has_swimming_pool               int64
has_garden                      int64
has_security                    int64
distance_to_city_center_km    float64
nearby_facilities_count         int64
dtype: object


Missing values:


listing_date                  0
city                          0
district                      0
property_type                 0
price_idr                     0
price_usd                     0
land_size_sqm                 0
building_size_sqm             0
bedrooms                      0
bathrooms                     0
floors                        0
building_age_years            0
carports                      0
certificate_type              0
furnishing                    0
has_swimming_pool             0
has_garden                    0
has_security                  0
distance_to_city_center_km    0
nearby_facilities_count       0
dtype: int64

In [5]:
# TODO: Add your preprocessing steps here
# Examples:
# - Drop unnecessary columns (id, listing_date, price_idr if using price_usd)
# - Handle missing values
# - Encode categorical variables
# - Handle outliers
# - Feature scaling

# Placeholder: After preprocessing, you should have X and y ready
# X should be your features (all columns except target)
# y should be your target variable (price_usd or log-transformed price)

## Data Splitting

We will detect the inputs and the outputs of the data which are X and y respectively.

Then we will split our data into the following:

- Training Size 70%
- Validation Size 15%
- Testing Size 15%

We will do this by splitting our data twice using the train_test_split function in sklearn. The function takes inputs, outputs and the testing size. After that we will print the training, validation and testing shapes and sizes.

In [ ]:
# IMPORTANT: After your preprocessing, replace these lines with your actual X and y
# For now, this is a placeholder showing the expected structure

# Example (replace with your preprocessed data):
# X = df_scaled.drop(columns=["price_usd"])  # or however you've named your target
# y = df_scaled["price_usd"]

# Convert to numpy arrays
X = np.array(X)  # Replace with your feature matrix
y = np.array(y)  # Replace with your target variable

In [ ]:
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the test dataset into test and validation
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Display the shapes of the datasets
print(f"Training set is: {X_train.shape[0]} rows which is {round(X_train.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0]),4)*100}%")
print(f"Validation set is: {X_val.shape[0]} rows which is {round(X_val.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0]),4)*100}%")
print(f"Testing set is: {X_test.shape[0]} rows which is {round(X_test.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0]),4)*100}%")

print(f"\nFeature dimensions: {X_train.shape[1]}")

## Dataset Object

Now we will create the dataset object. This part is not complex but it's a bit tricky. We need to convert our dataset to PyTorch Dataset object as it will be more efficient during training. You can use the dataset as is, but let's keep things professional and efficient. 

First we define our class that will be taking the main Dataset class with the concept of inheritance. There is a big class in PyTorch, this class has several functions inside it, we will recreate that class and modify some functions to match our needs.

In the cell below, we rebuilt the constructor function which is `__init__`. We put X and y as parameters to this function which are the inputs and outputs respectively, then inside the function we define the inputs and convert them to tensors, then convert the outputs to tensors and we make the numbers as `float32`. Additionally, we moved all our data to the cuda device. Then we modified the `__len__` and the `__getitem__` to match our needs which gets the specific length/shape of the data, and the data of specific row in our data respectively.

**Important difference from classification**: For regression, we don't need to convert y to long integers - we keep it as float32 since we're predicting continuous values, not classes.

In [ ]:
# Create the dataset object for regression
class RegressionDataset(Dataset):

    # Constructor function
    def __init__(self, X, y):
        # Convert to tensors (pytorch only takes tensors) and move to device
        self.X = torch.tensor(X, dtype=torch.float32).to(device) 
        # For regression, y should be float32, not long (unlike classification)
        self.y = torch.tensor(y, dtype=torch.float32).to(device) 
    
    # Get the length of the data
    def __len__(self):
        return len(self.X)

    # Get the data of specific index
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
# Create object for training, validation and testing
training_data = RegressionDataset(X_train, y_train)
validation_data = RegressionDataset(X_val, y_val)
testing_data = RegressionDataset(X_test, y_test)

## Training Hyperparameters

Now we are setting the training hyperparameters. We define some variables which are the batch size, number of training epochs, hidden neurons, and learning rate.

For regression tasks, you might want to experiment with:
- Different numbers of hidden layers
- Different activation functions (ReLU is common for hidden layers)
- Learning rate scheduling
- Different optimizers

In [ ]:
BATCH_SIZE = 64
EPOCHS = 50
HIDDEN_NEURONS_1 = 64  # First hidden layer
HIDDEN_NEURONS_2 = 32  # Second hidden layer
LR = 1e-3

## Data Loaders

This concept may seem complicated, but it's very easy. PyTorch provides a class called DataLoader which allows us to create objects of it to simplify the training.

DataLoader is an object that we can loop through it to train according to batches. When we start training, we loop through epochs. If you skip the batch size it means that the amount of training data in one batch is equal to the complete amount of training data, this method is not efficient and in most cases you need to train through using batches. DataLoader allows you to loop through the batches easily during the training. When you create a dataloader, you define the batch size and enable the shuffle to randomize the data and then you can loop through it in each epoch to train normally.

In the next cell, we define a dataloader for each of our data (training, validation and testing).

In [ ]:
# A dataloader is a class that loads the data in batches. 
# It's basically an object that we use to do the for loop during each epoch.

# Create DataLoader objects
training_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=False)
testing_dataloader = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Example of what the dataloader object looks like

# The dataloader returns batches of data in each iteration
# 'batch' contains the input features (X) for multiple samples
# 'label' contains the corresponding target values (y) for those samples
# Notice that each run of the for loop will return a different batch of data

for batch, label in training_dataloader:
    print("Batch shape (features):", batch.shape)  # [batch_size, num_features]
    print("First sample features:", batch[0])  
    
    print("="*100)

    print("Label shape (targets):", label.shape)  # [batch_size]
    print("First few target values (prices):", label[:5])  # Show first 5 prices
    break  # Only show one batch as an example

## Model Class

Creating a model in PyTorch seems not very straightforward in the beginning, but if you understand how machine learning and deep learning works, it will be easy for you to understand PyTorch structure easily.

In the next cell we define a new class `RegressionModel` that inherits from `nn.Module` like we did for the dataset. Remember, in simple terms, we want to redefine some functions in the class to match our needs. In the constructor which is `__init__`, then `super(RegressionModel, self).__init__()` calls the constructor of the parent class nn.Module to ensure it's properly initialized.

Then, we create our layers:
- **Input layer**: Takes the input features and transforms them to the first hidden layer
- **Hidden layers**: With ReLU activation for non-linearity
- **Output layer**: Produces a single continuous value (the predicted price)

**Key differences from classification:**
1. **No Sigmoid/Softmax at the end**: For regression, we output a raw continuous value, not a probability
2. **Output size is 1**: We're predicting a single continuous value (price)
3. **ReLU activation**: Commonly used in hidden layers for regression tasks

In the function `forward`, this function is the forward propagation of the model - how the data flows inside the model from the input to the output. This means we can control this completely. That's how PyTorch is so customizable!

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        # Call the parent class (nn.Module) constructor
        # This initializes all the PyTorch machinery needed for your model
        super(RegressionModel, self).__init__()

        # First hidden layer: takes input features and transforms them
        # input_size = number of input features (columns in your dataset)
        # HIDDEN_NEURONS_1 = number of neurons in the first hidden layer
        self.hidden1 = nn.Linear(input_size, HIDDEN_NEURONS_1)
        
        # Second hidden layer: takes output from first hidden layer
        self.hidden2 = nn.Linear(HIDDEN_NEURONS_1, HIDDEN_NEURONS_2)
        
        # Output layer: produces a single continuous value (the predicted price)
        # Unlike classification, we output 1 value, not a probability distribution
        self.output = nn.Linear(HIDDEN_NEURONS_2, 1)
        
        # ReLU activation function for hidden layers
        # ReLU(x) = max(0, x) - introduces non-linearity and prevents vanishing gradients
        # This is the most common activation for regression tasks
        self.relu = nn.ReLU()
        
        # Other activation options you could try for hidden layers:
        # self.leaky_relu = nn.LeakyReLU()  # Allows small negative values
        # self.elu = nn.ELU()  # Exponential Linear Unit, smoother than ReLU
        # self.tanh = nn.Tanh()  # Output range [-1, 1]
        # Note: For regression OUTPUT layer, we typically use NO activation (linear output)

    def forward(self, x):
        # Define how data flows through the network (forward pass)
        # x starts as your input data (batch of samples with features)
        
        # Pass through first hidden layer and apply ReLU activation
        x = self.relu(self.hidden1(x))
        
        # Pass through second hidden layer and apply ReLU activation
        x = self.relu(self.hidden2(x))
        
        # Pass through output layer (NO activation here for regression)
        # This gives us a raw continuous value as the prediction
        x = self.output(x)
        
        # Return the predicted value (house price)
        return x

## Model Creation

Now let's create our model and move it to the assigned device (cuda if you have GPU or the CPU if you don't have any GPUs). Additionally, we will print a `summary` of the model using the summary function which will take our model and the input size.

In [ ]:
# Instantiate the neural network model
# RegressionModel() creates a new instance of our custom neural network class
# We pass the input size (number of features) to the constructor
# .to(device) moves the model to the appropriate device (GPU if available, otherwise CPU)
# This ensures all model parameters and computations happen on the correct hardware
model = RegressionModel(input_size=X_train.shape[1]).to(device)

# Display a summary of the model architecture
# torchinfo.summary() prints layer-by-layer details including:
# - Layer types and names
# - Output shapes at each layer
# - Number of trainable parameters (weights and biases)
# input_size specifies the input shape: a tuple with (batch_size, num_features)
# This helps verify the model structure and parameter count before training
summary(model, input_size=(1, X_train.shape[1]))

## Loss and Optimizer

In the next cell, we will create the loss function and the optimizer.

**Key difference from classification:**
- For **classification**, we used Binary Cross Entropy (BCE) loss
- For **regression**, we use Mean Squared Error (MSE) loss, which measures the average squared difference between predictions and actual values

The optimizer `Adam` will take the model parameters/weights and the learning rate, same as classification.

In [ ]:
# Define the loss function for regression
# MSELoss (Mean Squared Error Loss) calculates the average of squared differences
# between predictions and actual values: loss = mean((prediction - actual)^2)
# This penalizes larger errors more heavily than smaller ones
# 
# Other regression loss options:
# - nn.L1Loss() - Mean Absolute Error, less sensitive to outliers
# - nn.SmoothL1Loss() - Huber loss, combines MSE and MAE benefits
# - nn.HuberLoss() - Similar to SmoothL1Loss with configurable delta
criterion = nn.MSELoss()

# Define the optimizer to update model weights during training
# Adam (Adaptive Moment Estimation) adjusts learning rates automatically for each parameter
# model.parameters() gives all trainable weights/biases to optimize
# lr (learning rate) controls how big the weight updates are each step
# Other optimizer options:
# - torch.optim.SGD() - basic stochastic gradient descent, simpler but may need tuning
# - torch.optim.AdamW() - Adam with weight decay, better generalization
# - torch.optim.RMSprop() - good for RNNs
optimizer = Adam(model.parameters(), lr=LR)

## Training

Now comes the exciting part. As we mentioned several times, nothing is complex here if you know how deep learning works. You just need to know PyTorch syntax. We start by initializing the for loop with the specified number of epochs. But before that we define 4 lists and inside the loop we define another 4 variables.

### Variables:
* `total_loss_train`: here we will keep track of the training loss progress during each epoch, we calculate the loss during the batch and we use the loss value to optimize and modify the model parameters.

* `total_loss_val`: here we will keep track of the validation loss progress during each epoch, we calculate the loss during the batch to monitor for overfitting.

### Lists:
* `total_loss_train_plot`: We append the losses of the training to visualize them at the end.

* `total_loss_validation_plot`: We append the losses of the validation to visualize them at the end.

**Key differences from classification:**
1. We don't track accuracy (that's for classification). For regression, loss (MSE) is our main metric.
2. We can also track additional metrics like MAE (Mean Absolute Error) or R² score for better understanding.
3. No rounding or class comparison - we directly compare continuous predicted values with actual values.

### Training Process
Then, we start to loop through the training dataloaders. In the loop, we get our data from the data loader, and the inputs and labels are already on the device. We allow the model to make a prediction (forward propagation), then we get the output of the model and compare it with our original output using the loss criterion. We use the squeeze function to remove unnecessary dimensions. We add the loss amount to `total_loss_train`. 

Then we do `batch_loss.backward()` which performs backpropagation, we use the optimizer to update the weights using `optimizer.step()`, and then we reset the optimizer gradients using `optimizer.zero_grad()` which is a very important step.

### Validation
After that we exit the batch loop (train dataloader loop) and we start with the validation. Don't forget that we are still in the same epoch. Inside that we start with `torch.no_grad()` which means that we need the model to do predictions without being trained. We just need to see the validation performance. Then we do the same steps which are predicting and calculating loss.

At the end we print after each epoch the epoch number, training loss, and validation loss. We use the printing of "=" signs just for making the printing output look clean.

In [ ]:
# Initialize empty lists to store metrics for each epoch (used for plotting later)
total_loss_train_plot = []  # Stores training loss per epoch
total_loss_validation_plot = []  # Stores validation loss per epoch

# Main training loop - runs for the specified number of epochs
for epoch in range(EPOCHS):
    # Reset metrics at the start of each epoch
    total_loss_train = 0  # Accumulates loss during training
    total_loss_val = 0  # Accumulates loss during validation
    
    # Set model to training mode (enables dropout, batch norm updates, etc.)
    model.train()

    ## Training
    # Loop through all batches in the training dataset
    for data in training_dataloader:

        # Unpack the batch into inputs (features) and labels (target prices)
        inputs, labels = data

        # Forward pass: feed inputs through the model to get predictions
        # squeeze(1) removes dimension 1 to match label shape (e.g., [64,1] -> [64])
        prediction = model(inputs).squeeze(1)

        # Calculate loss: Mean Squared Error between predictions and actual prices
        batch_loss = criterion(prediction, labels)

        # Add this batch's loss to the running total
        # .item() converts the tensor to a Python number
        total_loss_train += batch_loss.item()

        # Backward pass: calculate gradients (how to adjust weights)
        batch_loss.backward()
        
        # Update model weights based on calculated gradients
        optimizer.step()
        
        # Clear gradients for next iteration (PyTorch accumulates them by default)
        optimizer.zero_grad()

    ## Validation
    # Set model to evaluation mode (disables dropout, batch norm in eval mode, etc.)
    model.eval()
    
    # Disable gradient calculation for validation (saves memory, speeds up computation)
    with torch.no_grad():
        # Loop through all batches in the validation dataset
        for data in validation_dataloader:
            # Unpack batch
            inputs, labels = data

            # Forward pass: get predictions (same as training)
            prediction = model(inputs).squeeze(1)

            # Calculate loss (same as training)
            batch_loss = criterion(prediction, labels)

            # Accumulate validation loss
            total_loss_val += batch_loss.item()

    # After both training and validation, store metrics for this epoch
    # Calculate average loss per batch
    avg_train_loss = total_loss_train / len(training_dataloader)
    avg_val_loss = total_loss_val / len(validation_dataloader)
    
    total_loss_train_plot.append(avg_train_loss)
    total_loss_validation_plot.append(avg_val_loss)

    # Print epoch summary: shows progress of training and validation
    print("="*70)
    print(f"Epoch {epoch + 1:3d}")
    print("-"*70)
    print(f"  Train      - Loss (MSE): {avg_train_loss:12.4f}")
    print(f"  Validation - Loss (MSE): {avg_val_loss:12.4f}")
    print("="*70)

## Testing

Now in this section, we will be testing our model. We will start the code with `torch.no_grad()` which means that we are telling PyTorch that we don't want to train the model, we will be using it only for testing. Then we will declare initial loss as zero and empty lists for predictions and actual values.

We will start by looping through the testing dataloader like we did before during training. Inside the loop, we get our data and we run our model on the data to get the predictions. We calculate the loss and add it to our overall loss. We also collect predictions and actual values to calculate additional regression metrics.

**Key differences from classification:**
- Instead of accuracy, we calculate regression metrics: MSE, RMSE (Root Mean Squared Error), MAE (Mean Absolute Error), MAPE (Mean Absolute Percentage Error), and R² score
- R² score tells us the proportion of variance explained by our model (1.0 is perfect, 0.0 means the model is no better than predicting the mean)
- MAPE gives us the average percentage error, making it easy to interpret (e.g., 10% means predictions are off by 10% on average)
- We compare continuous predicted values directly with actual values, no rounding needed

In [ ]:
# Set model to evaluation mode
model.eval()

# Disable gradient calculation for testing (no backpropagation needed)
with torch.no_grad():
    # Initialize testing metrics
    total_loss_test = 0
    all_predictions = []  # Store all predictions
    all_actuals = []  # Store all actual values
    
    # Loop through all batches in the testing dataset
    for data in testing_dataloader:
        # Unpack batch
        inputs, labels = data

        # Forward pass: get predictions
        prediction = model(inputs).squeeze(1)

        # Calculate loss for this batch
        batch_loss_test = criterion(prediction, labels)
        
        # Accumulate testing loss
        total_loss_test += batch_loss_test.item()
        
        # Store predictions and actual values for metrics calculation
        all_predictions.extend(prediction.cpu().numpy())
        all_actuals.extend(labels.cpu().numpy())

# Calculate average test loss
avg_test_loss = total_loss_test / len(testing_dataloader)

# Convert lists to numpy arrays for sklearn metrics
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

# Calculate regression metrics
mse = mean_squared_error(all_actuals, all_predictions)
rmse = np.sqrt(mse)  # Root Mean Squared Error
mae = mean_absolute_error(all_actuals, all_predictions)  # Mean Absolute Error
mape = np.mean(np.abs((all_actuals - all_predictions) / all_actuals)) * 100  # Mean Absolute Percentage Error
r2 = r2_score(all_actuals, all_predictions)  # R² score

# Print final testing metrics
print("="*70)
print("TESTING RESULTS")
print("="*70)
print(f"Mean Squared Error (MSE):        {mse:12.4f}")
print(f"Root Mean Squared Error (RMSE):  {rmse:12.4f}")
print(f"Mean Absolute Error (MAE):       {mae:12.4f}")
print(f"Mean Absolute Percentage Error:  {mape:12.4f}%")
print(f"R² Score:                        {r2:12.4f}")
print("="*70)
print("\nInterpretation:")
print(f"- On average, predictions are off by ${mae:,.2f} (MAE)")
print(f"- On average, predictions are off by {mape:.2f}% (MAPE)")
print(f"- The model explains {r2*100:.2f}% of the variance in house prices (R²)")

## Additional Testing Analysis

Let's create some visualizations to better understand our model's performance:

In [ ]:
# Create a figure with multiple subplots for analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Predicted vs Actual values
axes[0].scatter(all_actuals, all_predictions, alpha=0.5)
axes[0].plot([all_actuals.min(), all_actuals.max()], 
             [all_actuals.min(), all_actuals.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Prices')
axes[0].set_ylabel('Predicted Prices')
axes[0].set_title('Predicted vs Actual House Prices')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals (errors) distribution
residuals = all_actuals - all_predictions
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Zero Error')
axes[1].set_xlabel('Residuals (Actual - Predicted)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Prediction Errors')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nResidual Statistics:")
print(f"Mean of residuals: {residuals.mean():.4f} (should be close to 0)")
print(f"Std of residuals: {residuals.std():.4f}")

## Plotting Training Progress

Let's visualize how the training and validation loss changed over epochs. This helps us understand if the model learned properly and if there's any overfitting.

**What to look for:**
- Both training and validation loss should decrease over time
- If validation loss starts increasing while training loss decreases, that's overfitting
- If both losses plateau, the model has converged

In [ ]:
# Create a plot for training and validation loss
plt.figure(figsize=(10, 6))

plt.plot(total_loss_train_plot, label='Training Loss', linewidth=2)
plt.plot(total_loss_validation_plot, label='Validation Loss', linewidth=2)
plt.title('Training and Validation Loss over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Save Trained Model

Now let's save our trained model so we can use it later without retraining.

In [ ]:
# Save the model state dict (recommended approach)
torch.save(model.state_dict(), 'house_price_regression_model.pth')

# Optionally save additional information for reproducibility
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'input_size': X_train.shape[1],
    'hidden_neurons_1': HIDDEN_NEURONS_1,
    'hidden_neurons_2': HIDDEN_NEURONS_2,
    'final_train_loss': total_loss_train_plot[-1],
    'final_val_loss': total_loss_validation_plot[-1],
    'test_mse': mse,
    'test_r2': r2
}, 'house_price_regression_checkpoint.pth')

print("Model saved successfully!")
print("\nFiles created:")
print("  - house_price_regression_model.pth (model weights only)")
print("  - house_price_regression_checkpoint.pth (full checkpoint with metadata)")

print("\nTo load the model later:")
print("  model = RegressionModel(input_size=X.shape[1]).to(device)")
print("  model.load_state_dict(torch.load('house_price_regression_model.pth'))")
print("  model.eval()")

## Inference

Now let's see how we can use the model in inference mode. Inference means making predictions like you are in production.

**Important**: You need to preprocess your input data the same way you preprocessed your training data:
- Apply the same scaling/normalization
- Encode categorical variables the same way
- Use the same feature order

In [ ]:
# Example: Make a prediction for a new house
# You need to prepare your input with the same preprocessing as training data

# Let's use an example from the test set to demonstrate
sample_idx = 0  # First sample from test set
sample_features = X_test[sample_idx]
actual_price = y_test[sample_idx]

# Convert to tensor and add batch dimension
model_input = torch.tensor(sample_features, dtype=torch.float32).unsqueeze(0).to(device)

# Make prediction
model.eval()
with torch.no_grad():
    predicted_price = model(model_input).item()

print("="*70)
print("INFERENCE EXAMPLE")
print("="*70)
print(f"Actual Price:    ${actual_price:,.2f}")
print(f"Predicted Price: ${predicted_price:,.2f}")
print(f"Difference:      ${abs(actual_price - predicted_price):,.2f}")
print(f"Error Percentage: {abs(actual_price - predicted_price) / actual_price * 100:.2f}%")
print("="*70)

In [ ]:
# For a completely new house, you would do something like this:
# (This is pseudocode - adjust based on your actual preprocessing)

# new_house = {
#     'land_size_sqm': 200,
#     'building_size_sqm': 150,
#     'bedrooms': 3,
#     'bathrooms': 2,
#     'floors': 2,
#     'building_age_years': 5,
#     'carports': 1,
#     'city': 'Jakarta',  # Need to encode
#     'property_type': 'House',  # Need to encode
#     # ... other features
# }

# Steps:
# 1. Encode categorical variables (same encoding as training)
# 2. Apply the same scaling/normalization (using the same scaler from training)
# 3. Arrange features in the same order as training data
# 4. Convert to tensor and make prediction

print("To make predictions on new data:")
print("1. Preprocess the data exactly as you did for training")
print("2. Convert to tensor: torch.tensor(data, dtype=torch.float32).to(device)")
print("3. Make prediction: model(tensor).item()")
print("4. If you applied log transform to prices, remember to inverse transform!")

## Summary and Next Steps

Congratulations! You've successfully built a PyTorch regression model for house price prediction.

**What you learned:**
1. How to structure a regression problem (continuous output vs. classification)
2. The key differences in PyTorch implementation:
   - Using MSE loss instead of BCE loss
   - No sigmoid/softmax activation on the output layer
   - Evaluating with regression metrics (MSE, RMSE, MAE, R²) instead of accuracy
3. How to train, validate, and test a regression model
4. How to visualize predictions and errors

**To improve your model, you could try:**
1. Adding more hidden layers or neurons
2. Using different activation functions
3. Implementing learning rate scheduling
4. Adding dropout or batch normalization for regularization
5. Trying different optimizers (AdamW, SGD with momentum)
6. Feature engineering in the preprocessing stage
7. Experimenting with different batch sizes and number of epochs
8. Using early stopping to prevent overfitting

Feel free to experiment with the hyperparameters and architecture!